# GBM-LCCM Example
The purpose of this notebook is to demonstrate how the Gaussian Bernoulli - Latent Class Choice Model (GBM-LCCM) code from Sfeir et al. (2020) could be used.

The application is based on the "London Dataset" from Hillel et al.(2018) which is available online as a supplementary material to the paper.

The dataset contains individual trip diaries of the London Travel Demand Survey (LTDS) from April 2012 to March 2015 with their corresponding modes alternatives extracted from a Google directions application programming interface (API) and corresponding estimates of car operating costs and public transport fares. The dataset consists of 81,086 trips, four modes (walke, cycle, public transport, and drive), and different trip purposes (e.g. Home-Based Work, Home-Based Education, etc.).

The required steps to build a travel mode choie model and run the GBM-LCCM code successfully are listed below.

##### References
G. Sfeir, M. Abou-Zeid, F. Rodrigues, F. C. Pereira, and I. Kaysi, “Semi-nonparametric Latent Class Choice Model with a Flexible Class Membership Component: A Mixture Model Approach,” arXiv Prepr. arXiv2007.02739, 2020.

T. Hillel, M.Z.E.B., Elshafie, and Y. Jin, "Recreating passenger mode choice-sets for transport
simulation: A case study of London, UK," in: Proceedings of the Institution of Civil Engineers - Smart Infrastructure and Construction, 2018

### Step 1: Import the GBM-LCCM Package and All other Necessary Packages

In [1]:
import GBM_LCCM
import numpy as np
import pandas as pd
import pylogit as pl
import warnings
from collections import OrderedDict
from sklearn import preprocessing

### Step 2: Load the Dataset
The GBM-LCCM consists of two sub-models, a class memberhsip model and a class-specific choice model:

- The class membership model is defined as a Gaussian Mixture Model to cluster individuals probabilistically to K clusters (latent classes)

- Conditioned on the class membership of an individual, the class-specific choice model estimates the probability of choosing a specific alternative

The class-specific choice model needs the data to be in "long" format while the class membership model requires a "wide" format.

For more details regarding how to convert your data from wide to long format, you can refer to the following link: https://github.com/timothyb0912/pylogit/blob/master/examples/notebooks/Main%20PyLogit%20Example.ipynb

In [2]:
# load the "london" dataset, which is available in wide format
wide_london = pd.read_csv(open('.../Data/' + 'dataset.csv','rb'))

# the .copy() returns a copy of the dataset and ensures that modifications are made to a copy and not the original data
wide_london = wide_london.copy()

In [3]:
# for this application we will only use Home-Based Work (HBW) trips
wide_london = wide_london[wide_london.purpose == "HBW"]

In [4]:
# look at the first 5 rows of the data
wide_london.head(5).T

# Note that 5 columns are added to the original dataset:
# - travel_mode: 1 for walk, 2 for cycle, 3 for pt, and 4 for drive
# - walk_av, cycle_av, pt_av, drive_av: to determine the availability of each mode (1 if available and 0 otherwise).
#   For simplicity, we assume that all modes are available for all individuals

,17,125,139,140,141
trip_id,17,125,139,140,141
household_id,3,26,29,29,29
person_n,1,0,0,0,1
trip_n,1,6,0,1,0
purpose,HBW,HBW,HBW,HBW,HBW
fueltype,Average_Car,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car
faretype,full,free,full,full,full
bus_scale,1,0,1,1,1
survey_year,1,1,1,1,1
travel_year,2012,2012,2012,2012,2012


### Step 3: Convert the data from wide to long format
We will rely in this section on the pylogit package which was developed to estimate many discrete choice models such as multinomial logit models, mixed logit and nested logit https://pypi.org/project/pylogit/. The package can be also used to convert the data from wide to long format.

In [5]:
# look at the columns of the london dataset
wide_london.columns

Index([u'trip_id', u'household_id', u'person_n', u'trip_n', u'purpose',
       u'fueltype', u'faretype', u'bus_scale', u'survey_year', u'travel_year',
       u'travel_month', u'travel_date', u'day_of_week', u'start_time_linear',
       u'age', u'female', u'driving_license', u'car_ownership', u'distance',
       u'driving_traffic_percent', u'dur_walking', u'dur_cycling',
       u'dur_pt_total', u'dur_pt_access', u'dur_pt_rail', u'dur_pt_bus',
       u'dur_pt_int_total', u'dur_pt_int_waiting', u'dur_pt_int_walking',
       u'pt_n_interchanges', u'dur_driving', u'cost_transit',
       u'cost_driving_total', u'cost_driving_fuel', u'cost_driving_con_charge',
       u'travel_mode_', u'travel_mode', u'walk_av', u'cycle_av', u'pt_av',
       u'drive_av'],
      dtype='object')

In [6]:
# Create the list of individual specific variables (variables that are specific to an individual 
# and do not vary across alternatives)
ind_variables = wide_london.columns.tolist()[:20]


# Specify the variables that vary across individuals and some or all alternatives
# The keys are the column names that will be used in the long format dataframe
# The values are dictionaries whose key-value pairs are the alternative id and
# the column name of the corresponding column that encodes that variable for
# the given alternative. Examples below.
alt_varying_variables = {u'travel_time': dict([(1, 'dur_walking'),
                                               (2, 'dur_cycling'),
                                               (3, 'dur_pt_total'),
                                               (4, 'dur_driving')]),
                        u'travel_cost': dict([(3, 'cost_transit'),
                                              (4, 'cost_driving_total')])}

# Specify the availability variables
# Note that the keys of the dictionary are the alternative id's.
# The values are the columns denoting the availability for the
# given mode in the dataset.
availability_variables = {1: 'walk_av',
                          2: 'cycle_av',
                          3: 'pt_av',
                          4: 'drive_av'}


# Determine the columns for: alternative ids, the observation ids and the choice
# The 'custom_alt_id' is the name of a column to be created in the long-format data
# It will identify the alternative associated with each row.
custom_alt_id = "mode_id"

# Create a custom id column that ignores the fact that this is a 
# panel/repeated-observations dataset. Note the +1 ensures the id's start at one.
obs_id_column = "custom_id"
wide_london[obs_id_column] = np.arange(wide_london.shape[0],
                                            dtype=int) + 1


# Create a variable recording the choice column
choice_column = "travel_mode"

In [7]:
# Perform the conversion to long-format
long_london = pl.convert_wide_to_long(wide_london, 
                                           ind_variables, 
                                           alt_varying_variables, 
                                           availability_variables, 
                                           obs_id_column, 
                                           choice_column,
                                           new_alt_id_name=custom_alt_id)
# Look at the resulting long-format dataframe
long_london.head(10).T

C:\Users\gms12\.conda\envs\logitenv\lib\site-packages\pylogit\choice_tools.py:453: UserWarning: Note, there are 42 variables in wide_data but the inputs ind_vars, alt_specific_vars, and subset_specific_vars only account for 32 variables.
  msg_2 + msg_3.format(num_vars_accounted_for))


,0,1,2,3,4,5,6,7,8,9
custom_id,1,1,1,1,2,2,2,2,3,3
mode_id,1,2,3,4,1,2,3,4,1,2
travel_mode,0,0,1,0,0,0,0,1,0,0
trip_id,17,17,17,17,125,125,125,125,139,139
household_id,3,3,3,3,26,26,26,26,29,29
person_n,1,1,1,1,0,0,0,0,0,0
trip_n,1,1,1,1,6,6,6,6,0,0
purpose,HBW,HBW,HBW,HBW,HBW,HBW,HBW,HBW,HBW,HBW
fueltype,Average_Car,Average_Car,Average_Car,Average_Car,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car,Petrol_Car
faretype,full,full,full,full,free,free,free,free,full,full


### Step 4: Model Specification

#### Step 4.1: Class Membership Model

In [8]:
# set the number of clusters (latent classes) to 2
n_classes = 2

In [9]:
# create dummy variables for car ownership
wide_london['car_ownership0'] = (wide_london['car_ownership'] == 0).astype(int)
wide_london['car_ownership1'] = (wide_london['car_ownership'] == 1).astype(int)
wide_london['car_ownership2'] = (wide_london['car_ownership'] == 2).astype(int)

In [10]:
# the class membership model is defined as a Gaussian Bernoulli Mixture Model
# Gaussian Mixture Model is used for continuous variables while Bernoulli Mixture Model for discrete variables

# select the continuous variables for the class membership model
X_Mem_C = wide_london[['age']]
# it is a good practice to standardize the continuous variables for the Gaussian Mixture Model
Standardized_X_Mem_C = preprocessing.scale(X_Mem_C)

# select the discrete variables for the class membership model
X_Mem_D = wide_london[['female','driving_license','car_ownership1','car_ownership2']]
X_Mem_D = X_Mem_D.values

C:\Users\gms12\.conda\envs\logitenv\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64 were all converted to float64 by the scale function.
  import sys


#### Step 4.2: Class-Specific Choice Model
This section is based on the latent class choice model (lccm) package. For more details on how to specify the class-specific choice model, you can check the lccm package on the following link: https://github.com/ferasz/LCCM

You can use alternative specific or generic speficifation for the parameters. 
We will use in this example both types to demonstrate how each one can be used:

- intercept: alternative specific constants included in all the utilities except the one related to walk, hence the one braket with the numbers of the corresponding alternatives [2,3,4] (walk is used as a base)
- travel time parameters for both classes are specified as alternative specific and included in the utilities of all modes, hence the one braket with the numbers of the corresponding alternatives [1,2,3,4]
- travel cost parameters for both classes are specified as generic and included only in the utilities of drive and pt, hence the two brakets and the numbers of the alternatives [[3,4]]


The specification of the two classes is as follows:
##### Latent Class 1:

$V_{walk} =        \beta_{tt, class1} * TravelTime_{walk}  $  

$V_{cycle} = ASC-cycle_{class1} + \beta_{tt, class1} * TravelTime_{cycle}$

$V_{pt} = ASC-pt_{class1} + \beta_{tt, class1} * TravelTime_{pt} + \beta_{cost, class1} * TravelCost_{pt} $

$V_{drive} = ASC-drive_{class1} + \beta_{tt, class1} * TravelTime_{drive} + \beta_{cost, class1} * TravelCost_{drive} $

##### Latent Class 2:

$V_{walk} =        \beta_{tt, class2} * TravelTime_{walk}  $  

$V_{cycle} = ASC-cycle_{class2} + \beta_{tt, class2} * TravelTime_{cycle}$

$V_{pt} = ASC-pt_{class2} + \beta_{tt, class2} * TravelTime_{pt} + \beta_{cost, class2} * TravelCost_{pt} $

$V_{drive} = ASC-drive_{class2} + \beta_{tt, class2} * TravelTime_{drive} + \beta_{cost, class2} * TravelCost_{drive} $

In [11]:
# scale travel time from hours to minutes
long_london['travel_time'] = long_london['travel_time'] * 60

# add a column of ones named'intercept'. This will be used later to define the Alternative-Specific Constants (ASCs).
long_london['intercept']=np.ones(long_london.shape[0])

In [12]:
# NOTE: Specification and variable names must be in lists of ordered dictionaries.
# class_specific_specs defines the variables names to be used in the specification of the 
# class specific choice model of each class.
# class_specific_labels defines the names associated with each of the variables which
# will be displayed in the output tables after estimation.


class_specific_specs = [OrderedDict([('intercept',[2,3,4]),
                                     ('travel_time',[1,2,3,4]),
                                     ('travel_cost',[[3,4]])]),
                        OrderedDict([('intercept',[2,3,4]),
                                     ('travel_time',[1,2,3,4]),
                                     ('travel_cost',[[3,4]])])]
                       
class_specific_labels = [OrderedDict([('ASC',['ASC(cycle)',
                                             'ASC(pt)',
                                             'ASC(drive)',]),
                                      ('travel_time',['Travel Time(walk)',
                                                      'Travel Time(cycle)',
                                                      'Travel Time(pt)',
                                                      'Travel Time(drive)']),
                                      ('travel_cost',['Travel Cost'])]),
                         OrderedDict([('ASC',['ASC(cycle)',
                                             'ASC(pt)',
                                             'ASC(drive)',]),
                                      ('travel_time',['Travel Time(walk)',
                                                      'Travel Time(cycle)',
                                                      'Travel Time(pt)',
                                                      'Travel Time(drive)']),
                                      ('travel_cost',['Travel Cost'])])]

In [13]:
# Starting Values for the Class-Specific Choice Model paramerters
paramClassSpec = []
paramClassSpec.append(np.array([0,0,0,0,0,0,0,0])) # for the first class
paramClassSpec.append(np.array([0,0,0,0,0,0,0,0])) # for the second class

### Step 5: Model Estimation and Results

To train/estimate the model, the following must be included in the GBM_LCCM.lccm_fit function:
- data: data used for the class-speficic choice model in long format
- X: continuous variables used for the class membership model in wide format
- X_Dummy: discrete variables used for the class membership model in wide format

For prediction/test, the following must be included in the GBM_LCCM.lccm_fit function:
- dataTest = data for the class-speficic choice model in long format
- XTest = continuous variables used for the class membership model in wide format
- XTest_Dummy = discrete variables used for the class membership model in wide format

Note that after training your model you can test it on a different dataset (test dataset). For this example, we use the same one that was used in training. However, you can split the data into train and test datasets(e.g. 80%-20%), estimate the model on the train dataset (data, X, X_Dummy) and test the model on the test dataset (dataTest, XTest, XTest_Dummy). The model will generate the prediction accuracy in terms of LL at the end of the notebook (Predicted Log-Likelihood).

The remaining variables that must be included in the GBM_LCCM.lccm_fit function are:
- prediction_test: 'Yes' if you would like to test the model on a test dataset and 'No' otherwise
- GMM_Initialization: 'random' to initialize the clusters of the Gaussian Bernoulli Mixture models randomly 
or 'kmeans' to initinalize the clusters using the k-means clustering algorithms
- covariance_type: the covariance structure of Gaussian Mixture Models can have four different types:
    - full, tied, diagonal and spherical. For more details about each type, you can refer to the following link: 
    - https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html
- ind_id_col: name of column identifying the individual for each row of data 
- obs_id_col: name of column identifying the observation (choice scenario)
- choice_col: name of column identifying whether the alternative corresponding by a row was chosen or not
- n_classes: number of latent classes (clusters)
- paramClassSpec: starting values for the class-specific choice models. If paramClassSpec is None, the model will generate the starting values randomly



Finally the code will store 

In [14]:
# Define a location to store the resutls 
outputFilePath = '.../GBM LCCM Example/'

# In this example we will assume that eahc trip was done by different individual
# So, that is why ind_id_col is equal to 'trip_id'


# Train/Estimate the model
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    
    GBM_LCCM.lccm_fit(data = long_london,
                      X = Standardized_X_Mem_C,
                      X_Dummy = X_Mem_D,
                      dataTest = long_london,
                      XTest = Standardized_X_Mem_C,
                      XTest_Dummy = X_Mem_D,
                      prediction_test = 'Yes',
                      GMM_Initialization = 'random',
                      ind_id_col = 'trip_id', 
                      obs_id_col = 'custom_id',
                      alt_id_col = 'mode_id',
                      choice_col = 'travel_mode', 
                      n_classes = n_classes,
                      covariance_type = 'full',
                      reg_covar = 1e-6,
                      tol = 1e-3,
                      max_iter = 100,
                      class_specific_specs = class_specific_specs,
                      class_specific_labels = class_specific_labels,
                      #paramClassSpec = paramClassSpec,
                      outputFilePath = outputFilePath,
                      outputFileName = 'ModelResults')

Processing data
Initializing EM Algorithm...

<Tue, 02 Feb 2021 21:17:08> Iteration 0: -71489.4893
<Tue, 02 Feb 2021 21:17:08> Iteration 1: -62930.2752
<Tue, 02 Feb 2021 21:17:09> Iteration 2: -62269.9896
<Tue, 02 Feb 2021 21:17:10> Iteration 3: -61646.9685
<Tue, 02 Feb 2021 21:17:10> Iteration 4: -61236.0749
<Tue, 02 Feb 2021 21:17:11> Iteration 5: -61054.8979
<Tue, 02 Feb 2021 21:17:11> Iteration 6: -60985.1465
<Tue, 02 Feb 2021 21:17:12> Iteration 7: -60956.2952
<Tue, 02 Feb 2021 21:17:12> Iteration 8: -60943.1821
<Tue, 02 Feb 2021 21:17:12> Iteration 9: -60936.9616
<Tue, 02 Feb 2021 21:17:13> Iteration 10: -60933.9400
<Tue, 02 Feb 2021 21:17:14> Iteration 11: -60932.4119
<Tue, 02 Feb 2021 21:17:14> Iteration 12: -60931.5705
<Tue, 02 Feb 2021 21:17:14> Iteration 13: -60931.0367
<Tue, 02 Feb 2021 21:17:15> Iteration 14: -60930.6340
<Tue, 02 Feb 2021 21:17:15> Iteration 15: -60930.2810
<Tue, 02 Feb 2021 21:17:16> Iteration 16: -60929.9408
<Tue, 02 Feb 2021 21:17:16> Iteration 17: -609

-----------------------------------------------------------------------------------------
ASC(cycle)                                       -2.4190     0.1177   -20.5459     0.0000
ASC(pt)                                          -1.1989     0.1013   -11.8315     0.0000
ASC(drive)                                        0.1731     0.0644     2.6861     0.0072
Travel Time(walk)                                -0.0938     0.0017   -54.6075     0.0000
Travel Time(cycle)                               -0.1081     0.0034   -32.0110     0.0000
Travel Time(pt)                                  -0.0774     0.0022   -35.0307     0.0000
Travel Time(drive)                               -0.1308     0.0022   -60.3453     0.0000
Travel Cost                                      -0.1740     0.0092   -18.9569     0.0000
-----------------------------------------------------------------------------------------

Class 2 Model: 
-----------------------------------------------------------------------------------